In [1]:
import re

In [2]:
def register_input_process(
    text,
):
    pat = r"Register ([A-C]): (\d+)"
    matches = re.search(pat, text)
    return matches.group(1), int(matches.group(2))

In [ ]:
with open("data/day17/input.txt", "r") as file:
    input_raw = file.read()

input_raw = """Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

register_raw, opcodes_raw = input_raw.split("\n\n")

In [4]:
register_init = dict(
    [register_input_process(text) for text in register_raw.split("\n")]
)
register = register_init.copy()

In [5]:
instructions = [int(x) for x in opcodes_raw.split(" ")[1].split(",")]

In [6]:
combo_operand_func = {
    0: lambda: 0,
    1: lambda: 1,
    2: lambda: 2,
    3: lambda: 3,
    4: lambda: register["A"],
    5: lambda: register["B"],
    6: lambda: register["C"],
}

In [7]:
def op0(op_lit):
    register["A"] = register["A"] // 2 ** combo_operand_func[op_lit]()


def op1(op_lit):
    register["B"] = register["B"] ^ op_lit


def op2(op_lit):
    register["B"] = combo_operand_func[op_lit]() % 8


def op3(op_lit):
    if register["A"] != 0:
        return "jump"


def op4(op_lit):
    register["B"] = register["B"] ^ register["C"]


def op5(op_lit):
    return combo_operand_func[op_lit]() % 8


def op6(op_lit):
    register["B"] = register["A"] // 2 ** combo_operand_func[op_lit]()


def op7(op_lit):
    register["C"] = register["A"] // 2 ** combo_operand_func[op_lit]()

In [8]:
op_func = {0: op0, 1: op1, 2: op2, 3: op3, 4: op4, 5: op5, 6: op6, 7: op7}

In [9]:
def run(prog):
    ins_point = 0
    output = []
    while ins_point < len(prog):
        opcode, operand = prog[ins_point], prog[ins_point + 1]
        val = op_func[opcode](operand)
        if val == "jump":
            ins_point = operand
        else:
            if val is not None:
                output.append(val)
            ins_point += 2

    return output

## Tests

In [10]:
register["C"] = 9
opcode, operand = 2, 6
op_func[opcode](operand)
assert register["B"] == 1

register["A"] = 10
instructions_test = [5, 0, 5, 1, 5, 4]
output = run(instructions_test)
assert all([i == j for i, j in zip(output, (0, 1, 2))])

register["A"] = 2024
instructions_test = [0, 1, 5, 4, 3, 0]
output = run(instructions_test)
assert all([i == j for i, j in zip(output, (4, 2, 5, 6, 7, 7, 7, 7, 3, 1, 0))])
assert register["A"] == 0

register["B"] = 29
opcode, operand = 1, 7
op_func[opcode](operand)
assert register["B"] == 26

register["B"] = 2024
register["C"] = 43690
opcode, operand = 4, 0
op_func[opcode](operand)
assert register["B"] == 44354

## Part 1

In [11]:
register = register_init.copy()

In [12]:
print(",".join([str(x) for x in run(instructions)]))

4,0,4,7,1,2,7,1,6


## Part 2

Seems to require reverse engineering the problem being in some repetitive loop. I'm going to call it a day here.